In [1]:
import requests
from bs4 import BeautifulSoup
from nba_api.stats.endpoints import leaguedashteamstats, leaguedashteamshotlocations
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Create an instance of the LeagueDashTeamStats endpoint
team_shot_locations_connect = leaguedashteamshotlocations.LeagueDashTeamShotLocations()

# Retrieve the data
team_shot_locations_data = team_shot_locations_connect.get_data_frames()[0]  # Get the first DataFrame

team_shot_locations = pd.DataFrame(team_shot_locations_data)
team_shot_locations = team_shot_locations

teams = team_shot_locations['']['TEAM_NAME']

rim_data = team_shot_locations[['Restricted Area']]
rim_data = rim_data['Restricted Area'][['FGM', 'FGA', 'FG_PCT']]

short_mid_data = team_shot_locations[['In The Paint (Non-RA)']]
short_mid_data = short_mid_data['In The Paint (Non-RA)'][['FGM', 'FGA', 'FG_PCT']]

long_mid_data = team_shot_locations[['Mid-Range']]
long_mid_data = long_mid_data['Mid-Range'][['FGM', 'FGA', 'FG_PCT']]

corner_3_data = team_shot_locations[['Corner 3']]
corner_3_data = corner_3_data['Corner 3'][['FGM', 'FGA', 'FG_PCT']]

above_the_break_data = team_shot_locations[['Above the Break 3']]
above_the_break_data = above_the_break_data['Above the Break 3'][['FGM', 'FGA', 'FG_PCT']]

rim = pd.concat([teams, rim_data], axis=1).set_index('TEAM_NAME')
short_mid = pd.concat([teams, short_mid_data], axis=1).set_index('TEAM_NAME')
long_mid = pd.concat([teams, long_mid_data], axis=1).set_index('TEAM_NAME')
corner_3 = pd.concat([teams, corner_3_data], axis=1).set_index('TEAM_NAME')
above_the_break = pd.concat([teams, above_the_break_data], axis=1).set_index('TEAM_NAME')

above_the_break.sort_values('FGA', ascending=False)

,FGM,FGA,FG_PCT
TEAM_NAME,,,
Golden State Warriors,314,846,0.371
Boston Celtics,301,842,0.357
Sacramento Kings,291,789,0.369
Dallas Mavericks,293,773,0.379
San Antonio Spurs,257,742,0.346
Atlanta Hawks,276,729,0.379
Milwaukee Bucks,255,722,0.353
Indiana Pacers,264,721,0.366
Memphis Grizzlies,245,710,0.345


In [29]:
new = 'ATB_'

above_the_break.rename(columns={'FGM':f'{new}FGM', 'FGA':f'{new}FGA', 'FG_PCT':f'{new}FG_PCT'}, inplace = True)

above_the_break

,ATB_FGM,ATB_FGA,ATB_FG_PCT
TEAM_NAME,,,
Atlanta Hawks,276,729,0.379
Boston Celtics,301,842,0.357
Brooklyn Nets,254,709,0.358
Charlotte Hornets,208,573,0.363
Chicago Bulls,248,684,0.363
Cleveland Cavaliers,239,696,0.343
Dallas Mavericks,293,773,0.379
Denver Nuggets,249,674,0.369
Detroit Pistons,192,574,0.334


In [75]:
# Create an instance of the LeagueDashTeamStats endpoint
team_stats = leaguedashteamstats.LeagueDashTeamStats()

# Retrieve the data
team_stats_data = team_stats.get_data_frames()[0]  # Get the first DataFrame

# Grab the PPP necessary stats
ovr_PPP_data = team_stats_data[['TEAM_NAME', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'TOV']]
PPP_stats = ovr_PPP_data.copy()

# Calculate points for each team
PPP_stats['FG2_POINTS'] = (PPP_stats['FGM'] - PPP_stats['FG3M']) * 2
PPP_stats['FG3_POINTS'] = PPP_stats['FG3M'] * 3
PPP_stats['FT_POINTS'] = PPP_stats['FTM']

# Calculate total points scored by each team
PPP_stats['TOTAL_POINTS'] = PPP_stats['FG2_POINTS'] + PPP_stats['FG3_POINTS'] + PPP_stats['FT_POINTS']
PPP_stats = PPP_stats.set_index('TEAM_NAME')

League_Totals = PPP_stats.copy()
League_Totals = League_Totals.drop(['FG2_POINTS', 'FG3_POINTS', 'FT_POINTS'], axis=1)

# Calculate totals for all stats including points
totals = PPP_stats.select_dtypes(np.number).sum().rename('League Totals')
PPP_stats = pd.concat([PPP_stats, totals.to_frame().T])

# Calculate Points per Possession (PPP) for each team
total_poss = (PPP_stats['FGA'] + (0.44 * PPP_stats['FTA']) + PPP_stats['TOV'])
PPP_stats['PPP'] = round((PPP_stats['TOTAL_POINTS'] / (PPP_stats['FGA'] + (0.44 * PPP_stats['FTA']) + PPP_stats['TOV'])),2)
#PPP_stats[['PPP']].sort_values('PPP', ascending=False)

In [76]:
# Concat the Shot Zones to the league totals
League_Totals = pd.concat([League_Totals, rim, short_mid, long_mid, corner_3, above_the_break], axis=1)

# Calculate the league totals
totals = League_Totals.select_dtypes(np.number).sum().rename('League Totals')
League_Totals = pd.concat([League_Totals, totals.to_frame().T])

# Caculate the true FG_PCT for each shot zone
League_Totals['RIM_FG_PCT']['League Totals'] = League_Totals['RIM_FGM']['League Totals'] / League_Totals['RIM_FGA']['League Totals']
League_Totals['SM_FG_PCT']['League Totals'] = League_Totals['SM_FGM']['League Totals'] / League_Totals['SM_FGA']['League Totals']
League_Totals['LM_FG_PCT']['League Totals'] = League_Totals['LM_FGM']['League Totals'] / League_Totals['LM_FGA']['League Totals']
League_Totals['C3_FG_PCT']['League Totals'] = League_Totals['C3_FGM']['League Totals'] / League_Totals['C3_FGA']['League Totals']
League_Totals['ATB_FG_PCT']['League Totals'] = League_Totals['ATB_FGM']['League Totals'] / League_Totals['ATB_FGA']['League Totals']

# Calculate the frequency
League_Totals['RIM_FREQ'] = League_Totals['RIM_FGA'] / League_Totals['FGA']
League_Totals['SM_FREQ'] = League_Totals['SM_FGA'] / League_Totals['FGA']
League_Totals['LM_FREQ'] = League_Totals['LM_FGA'] / League_Totals['FGA']
League_Totals['C3_FREQ'] = League_Totals['C3_FGA'] / League_Totals['FGA']
League_Totals['ATB_FREQ'] = League_Totals['ATB_FGA'] / League_Totals['FGA']

In [78]:
League_Totals

,FGM,FGA,FG3M,FG3A,FTM,FTA,TOV,TOTAL_POINTS,RIM_FGM,RIM_FGA,RIM_FG_PCT,SM_FGM,SM_FGA,SM_FG_PCT,LM_FGM,LM_FGA,LM_FG_PCT,C3_FGM,C3_FGA,C3_FG_PCT,ATB_FGM,ATB_FGA,ATB_FG_PCT,RIM_FREQ,SM_FREQ,LM_FREQ,C3_FREQ,ATB_FREQ
Atlanta Hawks,1149.0,2433.0,371.0,982.0,522.0,625.0,359.0,3191.0,460.0,732.0,0.628000,197.0,457.0,0.431000,121.0,262.0,0.462000,95.0,249.0,0.382000,276.0,729.0,0.379000,0.300863,0.187834,0.107686,0.102343,0.299630
Boston Celtics,1062.0,2221.0,394.0,1059.0,423.0,527.0,344.0,2941.0,430.0,615.0,0.699000,135.0,311.0,0.434000,103.0,236.0,0.436000,93.0,213.0,0.437000,301.0,842.0,0.357000,0.276902,0.140027,0.106258,0.095903,0.379109
Brooklyn Nets,1117.0,2380.0,381.0,985.0,403.0,519.0,346.0,3018.0,445.0,686.0,0.649000,186.0,443.0,0.420000,105.0,266.0,0.395000,127.0,269.0,0.472000,254.0,709.0,0.358000,0.288235,0.186134,0.111765,0.113025,0.297899
Charlotte Hornets,1049.0,2235.0,281.0,784.0,395.0,495.0,349.0,2774.0,478.0,738.0,0.648000,190.0,443.0,0.429000,100.0,270.0,0.370000,73.0,201.0,0.363000,208.0,573.0,0.363000,0.330201,0.198210,0.120805,0.089933,0.256376
Chicago Bulls,1132.0,2521.0,344.0,934.0,468.0,596.0,333.0,3076.0,452.0,737.0,0.613000,159.0,401.0,0.397000,177.0,448.0,0.395000,96.0,247.0,0.389000,248.0,684.0,0.363000,0.292344,0.159064,0.177707,0.097977,0.271321
Cleveland Cavaliers,1129.0,2379.0,321.0,918.0,445.0,575.0,387.0,3024.0,512.0,786.0,0.651000,196.0,466.0,0.421000,100.0,209.0,0.478000,82.0,219.0,0.374000,239.0,696.0,0.343000,0.330391,0.195881,0.087852,0.092055,0.292560
Dallas Mavericks,1119.0,2365.0,410.0,1104.0,453.0,603.0,313.0,3101.0,417.0,592.0,0.704000,178.0,376.0,0.473000,114.0,293.0,0.389000,117.0,326.0,0.359000,293.0,773.0,0.379000,0.250317,0.158985,0.123890,0.137844,0.326850
Denver Nuggets,1237.0,2525.0,328.0,878.0,425.0,583.0,343.0,3227.0,537.0,820.0,0.655000,237.0,498.0,0.476000,135.0,329.0,0.410000,79.0,198.0,0.399000,249.0,674.0,0.369000,0.324752,0.197228,0.130297,0.078416,0.266931
Detroit Pistons,1107.0,2390.0,263.0,793.0,463.0,585.0,443.0,2940.0,516.0,813.0,0.635000,190.0,445.0,0.427000,138.0,339.0,0.407000,71.0,210.0,0.338000,192.0,574.0,0.334000,0.340167,0.186192,0.141841,0.087866,0.240167
Golden State Warriors,1064.0,2337.0,385.0,1039.0,500.0,633.0,405.0,3013.0,338.0,502.0,0.673000,194.0,460.0,0.422000,147.0,336.0,0.438000,71.0,188.0,0.378000,314.0,846.0,0.371000,0.214805,0.196834,0.143774,0.080445,0.362003


In [67]:
# Create an instance of the LeagueDashTeamStats endpoint
team_stats = synergyplaytypes.SynergyPlayTypes()

# Retrieve the data
team_stats_data = team_stats.get_data_frames() # Get the first DataFrame

In [68]:
team_stats_data

[Empty DataFrame
 Columns: [SEASON_ID, TEAM_ID, TEAM_ABBREVIATION, TEAM_NAME, PLAY_TYPE, TYPE_GROUPING, PERCENTILE, GP, POSS_PCT, PPP, FG_PCT, FT_POSS_PCT, TOV_POSS_PCT, SF_POSS_PCT, PLUSONE_POSS_PCT, SCORE_POSS_PCT, EFG_PCT, POSS, PTS, FGM, FGA, FGMX]
 Index: []]